### This is a script that sets up the QAQC portion of the 2015 WY Roads Update project.  It accomplishes the following:
* Randomly selects 5 24k quads for each 100k quad throughout the state.  This represents the areas where QAQC will take place.
* Clips the working roads layers to the 24k quads.
* Creates point layers that will represent the location along each road where the QAQC assessment will take place.  A point is randomly placed on every road within the selected 24k quads.
* The random point layers are exported the their corresponding feature datasets within the working file geodatabase.

In [ ]:
import arcpy
import os

In [ ]:
#Set workspace, initiate empty list that will house our Feature Classes
workspace = <<path to working file geodatabase>>
fcs = []

#Walk the file geodatabase looking for the 100k quad Feature Classes we are interested in
walk = arcpy.da.Walk(workspace, datatype="FeatureClass")

for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        if filename.endswith('QuadBoundaries'):
            fcs.append(os.path.join(dirpath, filename))
            
fcs

In [ ]:
#Dissolve our Feature Classes so that they consist of just one feature, export to shapefile.  
#Add shapefiles to an empty list.
diss_shps = []
Dissovled_100kQuads = <<path to folder where Dissovled_100kQuads will be written>>

for fc in fcs:
    out_feature = os.path.join(Dissovled_100kQuads, fc.rsplit('\\', 1)[-1] + '.shp')
    diss_shps.append(out_feature)
    arcpy.Dissolve_management(fc, out_feature)
    
diss_shps

In [ ]:
# Set workspace, overwrite option, and variables.
arcpy.env.workspace = <<path to folder where Dissovled_100kQuads are>>
arcpy.env.overwriteOutput = True 
    
sel_quads = <<path to where Selected24kQuads.shp is>>
Clipped_24kQuads = <<path to where Clipped_24kQuads will be written>>
clip_shps = []

#Select the randomly selected quads that have their centroid within each of the dissolved shapefiles.  
#Export the selection to new shapefiles.
for shp in diss_shps:
    out_clip = os.path.join(Clipped_24kQuads, shp.rsplit('\\', 1)[-1].split('_')[0] + '_24kQuads.shp')
    clip_shps.append(out_clip)
    
    arcpy.MakeFeatureLayer_management(sel_quads, "sel_quads")
    
    selection = arcpy.SelectLayerByLocation_management("sel_quads", "HAVE_THEIR_CENTER_IN", shp, "", "NEW_SELECTION")
    
    #Ensure the correct amount of features are being selected.
    matchcount = int(arcpy.GetCount_management(selection)[0]) 
    print(matchcount)
    
    arcpy.CopyFeatures_management(selection, out_clip) 

In [ ]:
#Walk the file geodatabase looking for the road Feature Classes we are interested in
walk_rd = arcpy.da.Walk(workspace, datatype="FeatureClass")
fc_rd = []

for dirpath, dirnames, filenames in walk_rd:
    for filename in filenames:
        if 'Road' in filename:
            fc_rd.append(os.path.join(dirpath, filename))
            
roads = fc_rd[-5:]
roads

In [ ]:
#Clip the roads layers to the randomly selected 24k quads, then create a point shapefile that contains randomly generated
#points, one for each road.

rnd_pt_list = []
Clipped_Roads = << path to where Clipped_Roads will be written>>
Random_Points = << path to where Random_Points will be written>>

for rd in roads:
    out_clip_rd = os.path.join(Clipped_Roads, rd.rsplit('\\', 1)[-1].split('_')[0] + '_WYRoads_Clip.shp')
    arcpy.Clip_analysis(rd, sel_quads, out_clip_rd)
    arcpy.AddField_management(out_clip_rd, "Random", "SHORT")
    arcpy.CalculateField_management(out_clip_rd, "Random", "1", "PYTHON_9.3")
    rnd_pts = arcpy.CreateRandomPoints_management(out_path=Random_Points, 
                                    out_name=(out_clip_rd.rsplit('\\', 1)[-1].rsplit('_', 1)[0] + '_RandomPt'),
                                    constraining_feature_class=out_clip_rd, constraining_extent="0 0 250 250",
                                    number_of_points_or_field="Random", minimum_allowed_distance="50 Meters",
                                    create_multipoint_output="POINT", multipoint_size="0")
    rnd_pt_list.append(rnd_pts)
    
    print(out_clip_rd)

In [ ]:
#Randomly select half of the points to be assessed during QAQC.  This creates a more manageable, but still statistically
#significant subset.  Add fields that will be used to track errors and progress.

#Code block for input to Calculate Field operation.
codeblock = """
def rand_num():  
    import random  
    return random.randint(0,1)"""

for pt in rnd_pt_list:
    #Add fields
    arcpy.AddField_management(pt, "Random", "SHORT")
    arcpy.AddField_management(pt, "Attr_Err", "TEXT", field_length='1')
    arcpy.AddField_management(pt, "Pos_Err", "TEXT", field_length='1')
    arcpy.AddField_management(pt, "Om_Err", "TEXT", field_length='1')
    arcpy.AddField_management(pt, "Com_Err", "TEXT", field_length='1')
    arcpy.AddField_management(pt, "Assessed", "TEXT", field_length='1')
    #Randomly assign 0 or 1 to all values in 'Random' columns.  Only values with '1' will be assessed.
    arcpy.CalculateField_management(in_table=pt, field="Random", expression='rand_num()', expression_type="PYTHON_9.3",
                                    code_block=codeblock)

In [ ]:
#Set workspace, initiate empty list that will house our Feature Datasets
workspace = <<path to working file geodatabase>>
fdatasets = []


walk = arcpy.da.Walk(workspace, datatype="FeatureDataset")
for dirpath, dirnames, filenames in walk:
    if 'Block' in dirpath:
        fdatasets.append(dirpath)
        
#Sort feature dataset list alphabetically, so the items will correspond correctly to the list of point datasets.
sort = sorted(fdatasets)
sort

In [ ]:
# create a range [0,1,2] to index the lists
workRange = range(len(rnd_pt_list))

for thisIndex in workRange:
    # step through the lists one by one, and create a variable that will become the output name.
    shp = rnd_pt_list[thisIndex]
    fds = sort[thisIndex]
    temp = shp.rsplit('\\', 1)[-1].split('.')[0]
    # Export random point shapefiles to the corresponding feature datasets within the working file geodatabase.
    arcpy.FeatureClassToFeatureClass_conversion(shp, fds, temp)